<a href="https://colab.research.google.com/github/dmamur/elementsem/blob/main/element_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re, glob,os,sys,pickle,random
from collections import defaultdict

In [2]:
!pip install ase
!git clone https://github.com/dmamur/elementsem.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.7 MB/s eta 0:00:00
Cloning into 'elementsem'...
remote: Enumerating objects: 313, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 313 (delta 10), reused 0 (delta 0), pack-reused 282
Receiving objects: 100% (313/313), 74.19 MiB | 11.73 MiB/s, done.
Resolving deltas: 100% (123/123), done.
Updating files: 100% (233/233), done.


In [14]:
class Config:
    MAX_LEN = 128
    BATCH_SIZE = 32
    LR = 0.001
    VOCAB_SIZE = 128
    EMBED_DIM = 128
    NUM_HEAD = 4 # used in bert model
    FF_DIM = 128 # used in bert model
    NUM_LAYERS = 4
    MNAME = 'elembert_'
    MVER = 'V2'
    DSPATH="/content/elementsem/data/"
    PREPATH="/content/elementsem/models/pretrained/"
    PATH="/content/elementsem/models/"
config = Config()


# Load data from matbench

In [15]:
from ase.io import read,write
from scipy.ndimage import gaussian_filter1d
from ase.neighborlist import neighbor_list
from joblib import dump, load
from sklearn.cluster import KMeans
from sklearn import decomposition
from ase import Atoms

ellst=open(config.PATH+'pcakm/ellist.txt','r').read().split('\n')
print(ellst)

km = {i: load(config.PATH+'pcakm/'+i+'_kmeans.pkl') for i in ellst}
pca = {i: load(config.PATH+'pcakm/'+i+'_pca.pkl') for i in ellst}

config.VOCAB_SIZE = len(ellst)

def getRawInputs(types,atoms,x,v):
    i, d = neighbor_list('id', atoms, 10.0, self_interaction=False)
    rdfatoms,ntypes=[],[]
    for k,l in enumerate(atoms):
        el=types[k]
        y = np.zeros(100)
        dist = np.round(d[i==k]*10)
        a,b=np.unique(dist, return_counts=True)
        np.put(y,a.astype(int)-1,b)
        values=gaussian_filter1d(y/v,1)
        num = km[el].predict(pca[el].transform(np.nan_to_num([values],nan=0,posinf=0, neginf=0)))[0]
        ntypes.append(el+str(num))#el2id[el+str(num)]
    return ntypes

['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'Ra', 'Ac', 'Th', 'Pa', 'U', 'Np', 'Pu', 'Am', 'Cm', 'Bk', 'Cf', 'Rn']


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator KMeans from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator PCA from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [16]:
from ase.build import molecule
atoms = molecule('CH3CH2NH2')
#typesLstTest = getModelInputs(train_inputs) 

In [17]:
x= np.arange(0,10,0.1)
v = np.concatenate([[1],4*np.pi/3*(x[1:]**3 - x[:-1]**3)])
typesN=getRawInputs(atoms.get_chemical_symbols(),atoms,x,v)

In [18]:
print(typesN)

['C10', 'C13', 'N4', 'H5', 'H5', 'H5', 'H5', 'H5', 'H4', 'H4']
